# module imports and data preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install tensorflow==2.11.1

import tensorflow as tf
print(tf.__version__)
# tensorflow version used at development was 2.11.1, tf_io version: 0.31.0, tfp version: 0.19.0
# tensorflow on google colab upgraded to 2.12.0 at March 30. 2023, in middle of this project
# which introduced some compatibility issues, fixing the version at 2.11.1 have no issues.

import torch
print(torch.__version__)
import matplotlib
print(matplotlib.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 KB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 91.8 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.0
    Uninstalling tensorboard-data-server-0.7.0:
      Successfully uninstalled tensorboard-data-serve

2.11.1
2.0.0+cu118
3.7.1


In [ ]:
# Other imports
! pip install tensorflow_addons==0.19.0
! pip install tensorflow_io==0.31.0
! pip install tensorflow_probability==0.19.0
! pip install porespy==2.2.2

import os
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
from imutils import paths
from tqdm import tqdm
import tensorflow as tf
import tensorflow_addons as tfa
# import tensorflow_datasets as tfds
import tensorflow_io as tfio
import tensorflow_hub as hub
import tensorflow_probability as tfp

import numpy as np
import cv2
import pandas as pd
import seaborn as sns
from scipy import interpolate
from scipy.stats import wasserstein_distance
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity as cos
from sympy.utilities.iterables import multiset_permutations
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score, roc_auc_score, confusion_matrix, mean_absolute_error
from sklearn.model_selection import *
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KernelDensity as kde
from IPython.display import Image, display
import porespy
from edt import edt

tfpl = tfp.layers
tfd = tfp.distributions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.7 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.32.0
    Uninstalling tensorflow-io-gcs-filesystem-0.32.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.32.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Define model functions and image preprocessing functions

## image preprocessing functions

In [ ]:
# Image preprocessing utils
def parse_images_saxs(image_path):
    image_string = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=[sem_imageSize, sem_imageSize])

    return image

In [ ]:
# Image preprocessing utils
def parse_images_saxs_for_mapping_tf_ds(image_path):
    image_string = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=[sem_imageSize, sem_imageSize])

    return image, image

In [ ]:
# Image preprocessing utils
def parse_images_sem_TSNE(image_path):
    image_string = tf.io.read_file(image_path)
    image = tfio.experimental.image.decode_tiff(image_string)[:, :, :3]   # in the doc, it transforms tiff to 4 channels, with additional channel of opacity which is not needed.
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = image[:sem_original_x_length, :sem_original_y_length, :]
    image = image[:sem_imageSize, :sem_imageSize, :]

    return image

In [ ]:
# Image preprocessing utils
def parse_images_sem_no_crop(image_path):
    image_string = tf.io.read_file(image_path)
    image = tfio.experimental.image.decode_tiff(image_string)[:, :, :3]   # in the doc, it transforms tiff to 4 channels, with additional channel of opacity which is not needed.
    image = image[:sem_original_x_length, :sem_original_y_length, :]
    image = tfa.image.gaussian_filter2d(image, 5)
    image = cv2.cvtColor(image.numpy().astype('float32'), cv2.COLOR_BGR2GRAY)
    image = cv2.adaptiveThreshold(image.astype('uint8'), 1, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, adapt_window, 2)
    image = cv2.GaussianBlur(image,(5,5),0)

    return image

In [ ]:
# Image preprocessing utils
def parse_images_sem_crop(image_path, left, bottom):
    image_string = tf.io.read_file(image_path)
    image = tfio.experimental.image.decode_tiff(image_string)[:, :, :3]   # in the doc, it transforms tiff to 4 channels, with additional channel of opacity which is not needed.
    image = image[:sem_original_x_length, :sem_original_y_length, :]
    image = tfa.image.gaussian_filter2d(image, 5)
    image = cv2.cvtColor(image.numpy().astype('float32'), cv2.COLOR_BGR2GRAY)
    image = cv2.adaptiveThreshold(image.astype('uint8'), 1, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, adapt_window, 2)
    image = cv2.GaussianBlur(image,(5,5),0)
    image = image[left:left+sem_imageSize, bottom:bottom+sem_imageSize]

    return image

In [ ]:
# Image preprocessing utils
def parse_images_sem_crop_tf_ds(image, left, bottom):
    # left = 0
    # bottom = 0
    image = image[left:left+sem_imageSize, bottom:bottom+sem_imageSize, :]
    image_2d = image[:, :, 0]

    return image, image_2d

## functions for vae model building

In [ ]:
%cd "/content/drive/MyDrive/nanowire-morphology-classification-project"

/content/drive/.shortcut-targets-by-id/1qb_BrPe6oJ2Dg0E6xoFYETWtLHETrwQj/nanowire-morphology-classification-project


In [ ]:
def upsample(filters, size, apply_dropout=False):

  result = Sequential()
  result.add(
    Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer='he_normal',
                                    bias_initializer='he_normal'))

  result.add(BatchNormalization())

  if apply_dropout:
      result.add(Dropout(0.5))

  result.add(ReLU())

  return result

In [ ]:
# define loss functions and metrics for segmentation task
# coef functions are for batches of images, score functions are for a single image

def dice_coef_2d(y_true, y_pred, smooth=0.000001):
  
    beta = 1
    tp = tf.reduce_sum(y_true * y_pred, axis=[1,2])
    fp = tf.reduce_sum(y_pred, axis=[1,2]) - tp
    fn = tf.reduce_sum(y_true, axis=[1,2]) - tp

    score = tf.reduce_mean(((1 + beta ** 2) * tp + smooth) \
            / ((1 + beta ** 2) * tp + beta ** 2 * fn + fp + smooth))
    return score

def dice_coef_3d(y_true, y_pred, smooth=0.000001):
  
    beta = 1
    tp = tf.reduce_sum(y_true * y_pred, axis=[1,2,3])
    fp = tf.reduce_sum(y_pred, axis=[1,2,3]) - tp
    fn = tf.reduce_sum(y_true, axis=[1,2,3]) - tp

    score = tf.reduce_mean(((1 + beta ** 2) * tp + smooth) \
            / ((1 + beta ** 2) * tp + beta ** 2 * fn + fp + smooth))
    return score

def dice_score(y_true, y_pred, smooth=0.000001):
  y_true = y_true.astype('float32')
  y_pred = y_pred.astype('float32')
  beta = 1
  tp = tf.reduce_sum(y_true * y_pred, axis=[1, 2])
  fp = tf.reduce_sum(y_pred, axis=[1, 2]) - tp
  fn = tf.reduce_sum(y_true, axis=[1, 2]) - tp

  score = tf.reduce_mean(((1 + beta ** 2) * tp + smooth) \
          / ((1 + beta ** 2) * tp + beta ** 2 * fn + fp + smooth)).numpy()
  return score

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def iou_coef(y_true, y_pred, smooth=0.000001):

    beta = 1
    tp = tf.reduce_sum(y_true * y_pred, axis=[1,2])
    fp = tf.reduce_sum(y_pred, axis=[1,2]) - tp
    fn = tf.reduce_sum(y_true, axis=[1,2]) - tp

    score = tf.reduce_mean((tp + smooth) \
            / (tp + fn + fp + smooth))
    return score

def iou_score(y_true, y_pred, smooth=0.000001):
  y_true = y_true.astype('float32')
  tp = tf.reduce_sum(y_true * y_pred, axis=[1, 2])
  fp = tf.reduce_sum(y_pred, axis=[1, 2]) - tp
  fn = tf.reduce_sum(y_true, axis=[1, 2]) - tp

  score = tf.reduce_mean((tp + smooth) \
          / (tp + fn + fp + smooth)).numpy()
  return score

def iou_coef_loss(y_true, y_pred):
    return 1 - iou_coef(y_true, y_pred)

negative_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)

def softmax_loss(y_true, y_pred):
  softmax_loss = tf.nn.softmax_cross_entropy_with_logits(y_true, y_pred, axis=[1, 2, 3])
  return softmax_loss

In [ ]:
def saxs_encoder(shape, feature_extractor_model, num_features, prior, kl_weight):
  inputs = Input(shape)

  x = feature_extractor_model(inputs)

  x = Dense(num_features, activation='relu', kernel_initializer='random_normal')(x)
  
  # x = AveragePooling2D((2,2))(x)

  encoded_size = x.shape[-1]
  
  # options for distribution: IndependentNormal or MultivariateNormalTriL
  x = Dense(tfpl.IndependentNormal.params_size(encoded_size), activation=None)(x)
  x = tfpl.IndependentNormal(encoded_size, activity_regularizer=tfpl.KLDivergenceRegularizer(prior, weight=kl_weight))(x)
  return Model(inputs=inputs, outputs=x)

In [ ]:
def saxs_decoder(shape, num_filter, filter_size):
  inputs = Input(shape)

  x = inputs

  upsampling_block = [
    upsample(num_filter, filter_size),
    upsample(num_filter, filter_size),
    upsample(num_filter, filter_size),
    upsample(num_filter, filter_size),
    # upsample(num_filter, filter_size),
  ]

  # x = Reshape((1, 1, x.shape[-1]))(x)

  for up in upsampling_block:
    x = up(x)
  
  last = Conv2DTranspose(3, filter_size,    # number of class, size of input vector (batch_size, x, y, class)
                              strides=2,
                              padding='same',
                              bias_initializer='he_normal',
                              kernel_initializer='he_normal',
                              activation='sigmoid')       # (batch_size, resolution, resolution, 3)
                              # sigmoid activation transforms values to between 0 and 1
  
  x = last(x)
  # Bernoulli distribution for the recreated image, for it's binary. () means we don't want the third dimension. We want (imageSize, imageSize)
  # x = tfpl.IndependentBernoulli((), convert_to_tensor_fn=tfd.Distribution.mode)(x)

  return Model(inputs=inputs, outputs=x)

In [ ]:
def sem_encoder(shape, feature_extractor_model, num_features, prior, kl_weight):
  inputs = Input(shape)

  x = feature_extractor_model(inputs)

  x = Dense(num_features, activation='relu', kernel_initializer='random_normal')(x)
  
  # x = AveragePooling2D((2,2))(x)

  encoded_size = x.shape[-1]
  
  # options for distribution: IndependentNormal or MultivariateNormalTriL
  x = Dense(tfpl.IndependentNormal.params_size(encoded_size), activation=None)(x)
  x = tfpl.IndependentNormal(encoded_size, activity_regularizer=tfpl.KLDivergenceRegularizer(prior, weight=kl_weight))(x)
  return Model(inputs=inputs, outputs=x)

In [ ]:
def sem_decoder(shape, num_filter, filter_size):
  inputs = Input(shape)

  x = inputs

  upsampling_block = [
    upsample(num_filter, filter_size),
    upsample(num_filter, filter_size),
    upsample(num_filter, filter_size),
    upsample(num_filter, filter_size),
    # upsample(num_filter, filter_size),
  ]

  # x = Reshape((1, 1, x.shape[-1]))(x)

  for up in upsampling_block:
    x = up(x)
  
  last = Conv2DTranspose(num_filter, filter_size,    # size of input vector (batch_size, x, y, num_filter)
                              strides=2,
                              padding='same',
                              bias_initializer='he_normal',
                              kernel_initializer='he_normal',
                              activation=None)
                              # no activation here because we are use logits directly in the Bernoulli layer, 
                              # and the Bernoulli layer will act as an activation layer
  
  x = last(x)
  x = tf.reduce_mean(x, axis=-1, keepdims=True)

  # Bernoulli distribution for the recreated image, for it's binary. () means we don't want the third dimension. We want (imageSize, imageSize)
  x = tfpl.IndependentBernoulli((), convert_to_tensor_fn=tfd.Distribution.mode)(x)

  return Model(inputs=inputs, outputs=x)

In [ ]:
def get_pretrained_image_encoder_model(Resnet50_backbone, imagesize, hidden_1, hidden_2, hidden_3):
    base_model = Resnet50_backbone
    base_model.trainable = True
    inputs = Input((imagesize, imagesize, 3))
    h = base_model(inputs, training=True)
    h = GlobalAveragePooling2D()(h)

    projection_1 = Dense(hidden_1)(h)                                        
    projection_1 = Activation("relu")(projection_1)
    projection_1 = BatchNormalization(epsilon=0.001)(projection_1)
    projection_2 = Dense(hidden_2)(projection_1)
    projection_2 = Activation("relu")(projection_2)
    projection_2 = BatchNormalization(epsilon=0.001)(projection_2)
    projection_3 = Dense(hidden_3)(projection_2)
    projection_3 = BatchNormalization(epsilon=0.001)(projection_3)

    resnet_model = Model(inputs, projection_3)
    
    return resnet_model

# hyperparameters

In [ ]:
training_batch_size = 8

# cropped image size for both saxs and sem images
saxs_imageSize = 192
sem_imageSize = 192

train_ratio = 6 / 9

# size of the latent space for both autoencoders
latent_space_num_filters = 32

# sem image without the bottom part
sem_original_x_length = 830
sem_original_y_length = 1280

# window size for adaptive thresholding
adapt_window = 25

# pair vae training hyperparameters
method = 'barlow'

sim_weight = 5e-3
saxs_saxs_weight = 4e-2
saxs_sem_weight = 4e-1
sem_sem_weight = 4e-2
sem_saxs_weight = 1e0

# initiate trained SAXS and SEM VAE model and load SAXS and SEM images

In [ ]:
Resnet50_transfer_sem = tf.keras.applications.ResNet50(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=(sem_imageSize, sem_imageSize, 3), 
    pooling=None,
)

Resnet50_transfer_saxs = tf.keras.applications.ResNet50(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=(saxs_imageSize, saxs_imageSize, 3), 
    pooling=None,
)

Resnet50_transfer_sem.trainable = False
Resnet50_transfer_saxs.trainable = False

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
num_filter = 8
filter_size = 5
prior_mean = 0
prior_variance = 1
saxs_kl_weight = 1e-6

encoder_res = np.array([512])
encoder_batch_size = np.array([8])
random_seed_list = np.array([45])

In [ ]:
# this is the training block of vae_encoder and vae_decoder
resnet_model = get_pretrained_image_encoder_model(Resnet50_transfer_saxs, saxs_imageSize, 128, 64, 1024)
if encoder_res[0] == 224:
  resnet_model.load_weights('barlow_resnet_batch%i_project128_64_1024_seed%i.h5' % (encoder_batch_size[0], random_seed_list[0]))
else:
  resnet_model.load_weights('barlow_resnet_batch%i_project128_64_1024_res%i_seed%i.h5' % (encoder_batch_size[0], encoder_res[0], random_seed_list[0]))
trained_resnet50 = resnet_model.get_layer('resnet50')
trained_resnet50.trainable = False

saxs_encoded_size = [saxs_imageSize // 32, saxs_imageSize // 32, latent_space_num_filters]

# gaussian distribution as the prior distribution for the encoded latent space
saxs_prior = tfd.Independent(tfd.Normal(loc=tf.zeros(saxs_encoded_size) + prior_mean, scale=prior_variance), reinterpreted_batch_ndims=1)

vae_saxs_encoder = saxs_encoder((saxs_imageSize, saxs_imageSize, 3), trained_resnet50, latent_space_num_filters, saxs_prior, saxs_kl_weight)

vae_saxs_decoder = saxs_decoder(saxs_encoded_size, num_filter, filter_size)

saxs_vae = Model(inputs=vae_saxs_encoder.inputs, outputs=vae_saxs_decoder(vae_saxs_encoder.outputs[0]))

In [ ]:
num_filter = 32
filter_size = 15
prior_mean = 0
prior_variance = 1
sem_kl_weight = 1e-6

encoder_res = np.array([512])
encoder_batch_size = np.array([8])
random_seed_list = np.array([45])

In [ ]:
# this is the training block of vae_encoder and vae_decoder
resnet_model = get_pretrained_image_encoder_model(Resnet50_transfer_sem, sem_imageSize, 128, 64, 1024)
if encoder_res[0] == 224:
  resnet_model.load_weights('barlow_resnet_batch%i_project128_64_1024_seed%i.h5' % (encoder_batch_size[0], random_seed_list[0]))
else:
  resnet_model.load_weights('barlow_resnet_batch%i_project128_64_1024_res%i_seed%i.h5' % (encoder_batch_size[0], encoder_res[0], random_seed_list[0]))
trained_resnet50 = resnet_model.get_layer('resnet50')
trained_resnet50.trainable = False

sem_encoded_size = [sem_imageSize // 32, sem_imageSize // 32, latent_space_num_filters]

# gaussian distribution as the prior distribution for the encoded latent space
sem_prior = tfd.Independent(tfd.Normal(loc=tf.zeros(sem_encoded_size) + prior_mean, scale=prior_variance), reinterpreted_batch_ndims=1)

vae_sem_encoder = sem_encoder((sem_imageSize, sem_imageSize, 3), trained_resnet50, latent_space_num_filters, sem_prior, sem_kl_weight)

vae_sem_decoder = sem_decoder(sem_encoded_size, num_filter, filter_size)

sem_vae = Model(inputs=vae_sem_encoder.inputs, outputs=vae_sem_decoder(vae_sem_encoder.outputs[0]))

In [ ]:
sem_image_path = "/content/drive/MyDrive/TEM image datasets/2023-Sci-Adv-SAXS-SEM/SEM"
sem_image_file_list = list(paths.list_files(basePath=sem_image_path, validExts='tif'))
sem_image_file_list.sort()
sem_image_file_list = np.array(sem_image_file_list)
saxs_image_path = "/content/drive/MyDrive/TEM image datasets/2023-Sci-Adv-SAXS-SEM/SAXS"
saxs_image_file_list = list(paths.list_files(basePath=saxs_image_path, validExts='jpg'))
saxs_image_file_list.sort()
saxs_image_file_list = np.array(saxs_image_file_list)

sem_image_holder = np.zeros((len(sem_image_file_list), sem_imageSize, sem_imageSize, 3))
sem_image_2d_holder = np.zeros((len(sem_image_file_list), sem_imageSize, sem_imageSize))
saxs_image_holder = np.zeros((len(saxs_image_file_list), saxs_imageSize, saxs_imageSize, 3))
for i in range(len(saxs_image_file_list)):
        saxs_image_holder[i] = parse_images_saxs(saxs_image_file_list[i])

# Solo trained models performance assessment: 5 models trained with different selections of training images, and assessing with 10 random SEM crops.

In [ ]:
random_seed = np.linspace(42, 46, 5)
random_seed_for_crop = np.linspace(42, 51, 10).astype(int)

sem_mean_average_error = np.zeros((len(random_seed), len(random_seed_for_crop), len(sem_image_file_list)))
saxs_mean_average_error = np.zeros((len(random_seed), len(sem_image_file_list)))

# resolution = 192
x_two_point_corr = np.linspace(0.5, sem_image_2d_holder.shape[-1] / 2 - 0.5, int(sem_image_2d_holder.shape[-1] / 2))

for k in range(len(random_seed)):

  mother_directory = '/content/drive/MyDrive/ML projects/SAXS_SEM/solo_trained_models/'
  save_file_name = 'cropSize%i_windowSize%i_latentSize%i_trainSize%i_seed%i.h5' \
  % (saxs_imageSize, adapt_window, latent_space_num_filters, np.rint(len(sem_image_file_list) * train_ratio).astype(int), random_seed[k])

  vae_saxs_encoder.load_weights(mother_directory + 'saxs_encoder_' + save_file_name)
  vae_saxs_decoder.load_weights(mother_directory + 'saxs_decoder_' + save_file_name)
  vae_sem_encoder.load_weights(mother_directory + 'sem_encoder_' + save_file_name)
  vae_sem_decoder.load_weights(mother_directory + 'sem_decoder_' + save_file_name)

  saxs_reconstruction = saxs_vae(saxs_image_holder)[:].numpy()

  for m in range(sem_image_2d_holder.shape[0]):
    saxs_mean_average_error[k, m] = mean_absolute_error(np.reshape(saxs_reconstruction[m], (-1, 1)), np.reshape(saxs_image_holder[m], (-1, 1)))

  for j in range(len(random_seed_for_crop)):
      np.random.seed(random_seed_for_crop[j])
      tf.random.set_seed(random_seed_for_crop[j])

      rng = np.random.default_rng(seed=random_seed_for_crop[j])
      
      for i in range(len(sem_image_file_list)):
        sem_image_2d_holder[i] = parse_images_sem_crop(sem_image_file_list[i], rng.integers(0, sem_original_x_length - sem_imageSize), rng.integers(0, sem_original_y_length - sem_imageSize))
        sem_image_holder[i] = np.stack((sem_image_2d_holder[i], sem_image_2d_holder[i], sem_image_2d_holder[i]), axis=-1)

      sem_reconstruction = sem_vae(sem_image_holder)[:].numpy()

      # calculate 2-point correlation for true images

      true_two_point_corr = np.zeros((sem_image_2d_holder.shape[0], len(x_two_point_corr)))

      for m in range(sem_image_2d_holder.shape[0]):
        image = sem_image_2d_holder[m]
        # vector metrics: 2-point correlation function
        true_two_point_corr[m, :] = porespy.metrics.two_point_correlation(image).pdf
        # .probability is the normalized where the 0th point is normalized to 1, .pdf is raw, where the 0th point is the porosity
        # check porespy.metrics.two_point_correlation(image).distance when mismatch error occurs

      # calculate 2-point correlation for reconstructed images

      gen_two_point_corr = np.zeros((sem_image_2d_holder.shape[0], len(x_two_point_corr)))

      for m in range(sem_image_2d_holder.shape[0]):
        image = sem_reconstruction[m]
        # vector metrics: 2-point correlation function
        gen_two_point_corr[m, :] = porespy.metrics.two_point_correlation(image).pdf
        # .probability is the normalized where the 0th point is normalized to 1, .pdf is raw, where the 0th point is the porosity
        # check porespy.metrics.two_point_correlation(image).distance when mismatch error occurs

      for m in range(sem_image_2d_holder.shape[0]):
        sem_mean_average_error[k, j, m] = mean_absolute_error(gen_two_point_corr[m], true_two_point_corr[m])

mother_directory = '/content/drive/MyDrive/ML projects/SAXS_SEM/performance_assessment/'
save_file_name = 'cropSize%i_windowSize%i_latentSize%i_trainSize%i' \
  % (saxs_imageSize, adapt_window, latent_space_num_filters, np.rint(len(sem_image_file_list) * train_ratio).astype(int))
np.savez_compressed(mother_directory + 'solo_vae_' + save_file_name + '.npz', saxs_mae=saxs_mean_average_error, sem_mae=sem_mean_average_error)

# Pair trained models performance assessment: 5 models trained with different selections of training images, and assessing with 10 random SEM crops.

In [ ]:
random_seed = np.linspace(42, 46, 5)
random_seed_for_crop = np.linspace(42, 51, 10).astype(int)

sem_sem_mean_average_error = np.zeros((len(random_seed), len(random_seed_for_crop), len(sem_image_file_list)))
saxs_sem_mean_average_error = np.zeros((len(random_seed), len(random_seed_for_crop), len(sem_image_file_list)))
saxs_saxs_mean_average_error = np.zeros((len(random_seed), len(sem_image_file_list)))
sem_saxs_mean_average_error = np.zeros((len(random_seed), len(random_seed_for_crop), len(sem_image_file_list)))

# resolution = 192
x_two_point_corr = np.linspace(0.5, sem_image_2d_holder.shape[-1] / 2 - 0.5, int(sem_image_2d_holder.shape[-1] / 2))

for k in range(len(random_seed)):

  mother_directory = '/content/drive/MyDrive/ML projects/SAXS_SEM/pair_trained_models/'
  save_file_name = 'cropSize%i_windowSize%i_latentSize%i_trainSize%i_%s_%0.0e_%0.0e_%0.0e_%0.0e_%0.0e_seed%i.h5' \
  % (sem_imageSize, adapt_window, latent_space_num_filters, np.rint(len(sem_image_file_list) * train_ratio).astype(int), method, sim_weight, saxs_saxs_weight, saxs_sem_weight, sem_sem_weight, sem_saxs_weight, random_seed[k])

  vae_saxs_encoder.load_weights(mother_directory + 'saxs_encoder_' + save_file_name)
  vae_saxs_decoder.load_weights(mother_directory + 'saxs_decoder_' + save_file_name)
  vae_sem_encoder.load_weights(mother_directory + 'sem_encoder_' + save_file_name)
  vae_sem_decoder.load_weights(mother_directory + 'sem_decoder_' + save_file_name)

  saxs_saxs_recon = saxs_vae(saxs_image_holder)[:].numpy()

  for m in range(sem_image_2d_holder.shape[0]):
    saxs_saxs_mean_average_error[k, m] = mean_absolute_error(np.reshape(saxs_saxs_recon[m], (-1, 1)), np.reshape(saxs_image_holder[m], (-1, 1)))

  for j in range(len(random_seed_for_crop)):
      np.random.seed(random_seed_for_crop[j])
      tf.random.set_seed(random_seed_for_crop[j])

      rng = np.random.default_rng(seed=random_seed_for_crop[j])
      
      for i in range(len(sem_image_file_list)):
        sem_image_2d_holder[i] = parse_images_sem_crop(sem_image_file_list[i], rng.integers(0, sem_original_x_length - sem_imageSize), rng.integers(0, sem_original_y_length - sem_imageSize))
        sem_image_holder[i] = np.stack((sem_image_2d_holder[i], sem_image_2d_holder[i], sem_image_2d_holder[i]), axis=-1)

      sem_sem_recon = sem_vae(sem_image_holder)[:].numpy()

      # calculate 2-point correlation for true images

      true_two_point_corr = np.zeros((sem_image_2d_holder.shape[0], len(x_two_point_corr)))
      for m in range(sem_image_2d_holder.shape[0]):
        image = sem_image_2d_holder[m]
        true_two_point_corr[m, :] = porespy.metrics.two_point_correlation(image).pdf
 
      # calculate 2-point correlation for sem-sem reconstruction

      sem_sem_two_point_corr = np.zeros((sem_image_2d_holder.shape[0], len(x_two_point_corr)))
      for m in range(sem_image_2d_holder.shape[0]):
        image = sem_sem_recon[m]
        sem_sem_two_point_corr[m, :] = porespy.metrics.two_point_correlation(image).pdf

      for m in range(sem_image_2d_holder.shape[0]):
        sem_sem_mean_average_error[k, j, m] = mean_absolute_error(sem_sem_two_point_corr[m], true_two_point_corr[m])
      
      saxs_sem_recon = vae_sem_decoder(vae_saxs_encoder(saxs_image_holder))[:].numpy()

      # calculate 2-point correlation for saxs-sem reconstruction

      saxs_sem_two_point_corr = np.zeros((sem_image_2d_holder.shape[0], len(x_two_point_corr)))
      for m in range(sem_image_2d_holder.shape[0]):
        image = saxs_sem_recon[m]
        saxs_sem_two_point_corr[m, :] = porespy.metrics.two_point_correlation(image).pdf

      for m in range(sem_image_2d_holder.shape[0]):
        saxs_sem_mean_average_error[k, j, m] = mean_absolute_error(saxs_sem_two_point_corr[m], true_two_point_corr[m])

      # mean_absolute_error of sem to saxs reconstruction
      sem_saxs_recon = vae_saxs_decoder(vae_sem_encoder(sem_image_holder))[:].numpy()

      for m in range(sem_image_2d_holder.shape[0]):
        sem_saxs_mean_average_error[k, j, m] = mean_absolute_error(np.reshape(sem_saxs_recon[m], (-1, 1)), np.reshape(saxs_image_holder[m], (-1, 1)))

mother_directory = '/content/drive/MyDrive/ML projects/SAXS_SEM/performance_assessment/'
save_file_name = 'cropSize%i_windowSize%i_latentSize%i_trainSize%i' \
  % (saxs_imageSize, adapt_window, latent_space_num_filters, np.rint(len(sem_image_file_list) * train_ratio).astype(int))
np.savez_compressed(mother_directory + 'pair_vae_' + save_file_name + '.npz', \
                    saxs_saxs_mae=saxs_saxs_mean_average_error, sem_sem_mae=sem_sem_mean_average_error, saxs_sem_mae=saxs_sem_mean_average_error, sem_saxs_mae=sem_saxs_mean_average_error)